# Plot Fluorescence Data

Plot fluorescence profiles from cell tracks exported by TrackMate

We begin by importing the necessary packages:

In [ ]:
import random
import os

import matplotlib.pyplot as plt
import pandas as pd
import glob

Specify input directory:

In [ ]:
# Constants
POSITION = 0
NUM_TRACKS = 4  # Number of tracks to plot; set to None or 0 to plot all tracks
INPUT_DIR = './TrackMate_Outputs'
valid_input = False

if os.path.exists(INPUT_DIR):
    print(f'{INPUT_DIR} is a valid directory - well done you!')
    valid_input = True
else:
    print(f'{INPUT_DIR} does not exist - check that the path is correct')

Load the data and sort it:

In [ ]:
data = pd.DataFrame()
if valid_input:
    file_paths = glob.glob(f'{INPUT_DIR}/Pos{POSITION}/*_Pos{POSITION}_spots.csv')
    
    print(f'{len(file_paths)} valid CSV files found in {INPUT_DIR}')
    
    if len(file_paths) < 1:
        print(f'Are you sure {INPUT_DIR} is the folder that contains your TrackMate data?')

    # Read and preprocess data
    else:
        data = pd.read_csv(file_paths[0], skiprows=[1, 2, 3])
        data.sort_values(by=['TRACK_ID', 'FRAME'], ignore_index=True, inplace=True)
else:
    print(f'{INPUT_DIR} does not exist - check that the path is correct')

Extract the relevant columns:

In [ ]:
intensity_data = pd.DataFrame()
unique_tracks = []
if data.shape[0] > 0:
    intensity_data = data[['FRAME', 'TRACK_ID', 'MEAN_INTENSITY_CH2', 'MEAN_INTENSITY_CH3']]
    unique_tracks = intensity_data['TRACK_ID'].unique()
else:
    print('No valid data read from files in above cells.')

Select some tracks at random:

In [ ]:
selected_tracks = []
if len(unique_tracks) < 1:
    print('No data found in cells above.')
# Check NUM_TRACKS does not exceed available tracks
elif NUM_TRACKS is None or NUM_TRACKS <= 0:
    selected_tracks = unique_tracks
else:
    NUM_TRACKS = min(NUM_TRACKS, len(unique_tracks))
    selected_tracks = random.sample(list(unique_tracks), NUM_TRACKS)

Plot the selected fluorescence profiles

In [ ]:
if len(selected_tracks) > 0:
    # Determine layout for subplots
    num_cols = 2  # Number of columns for subplots
    num_rows = (len(selected_tracks) + num_cols - 1) // num_cols  # Calculate number of rows needed
    
    fig, axes = plt.subplots(num_rows, num_cols, figsize=(12, 6 * num_rows))
    axes = axes.flatten()  # Flatten in case of a grid, even if it's a single row or column
    
    for i, track in enumerate(selected_tracks):
        ax = axes[i]
        track_data = intensity_data[intensity_data['TRACK_ID'] == track]
        ax.scatter(track_data['FRAME'], track_data['MEAN_INTENSITY_CH2'], label=f'C2', alpha=0.7)
        ax.scatter(track_data['FRAME'], track_data['MEAN_INTENSITY_CH3'], label=f'C3', alpha=0.7)
        ax.set_title(f'Position {POSITION} - Track {track}')
        ax.set_xlabel('Frame')
        ax.set_ylabel('Intensity')
        ax.legend()
    
    # Hide any empty subplots
    for j in range(i + 1, len(axes)):
        fig.delaxes(axes[j])
    
    plt.show()
else:
    print('No data found in cells above - nothing to plot!')